In [1]:
num_encoder_tokens = 32

In [2]:
num_decoder_tokens  = 34

In [3]:
target_token_index = {'\t': 0,
 '\n': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 'A': 6,
 'C': 7,
 'D': 8,
 'E': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'g': 15,
 'h': 16,
 'i': 17,
 'j': 18,
 'k': 19,
 'l': 20,
 'm': 21,
 'n': 22,
 'o': 23,
 'p': 24,
 'q': 25,
 'r': 26,
 's': 27,
 't': 28,
 'u': 29,
 'v': 30,
 'w': 31,
 'y': 32,
 'z': 33}

In [4]:
input_token_index = {'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 'A': 4,
 'B': 5,
 'C': 6,
 'D': 7,
 'E': 8,
 'a': 9,
 'b': 10,
 'd': 11,
 'e': 12,
 'f': 13,
 'g': 14,
 'h': 15,
 'i': 16,
 'k': 17,
 'l': 18,
 'm': 19,
 'n': 20,
 'o': 21,
 'p': 22,
 'q': 23,
 'r': 24,
 's': 25,
 't': 26,
 'u': 27,
 'v': 28,
 'w': 29,
 'y': 30,
 'z': 31}

In [5]:
reverse_target_char_index = {0: '\t',
 1: '\n',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: 'A',
 7: 'C',
 8: 'D',
 9: 'E',
 10: 'a',
 11: 'b',
 12: 'c',
 13: 'd',
 14: 'e',
 15: 'g',
 16: 'h',
 17: 'i',
 18: 'j',
 19: 'k',
 20: 'l',
 21: 'm',
 22: 'n',
 23: 'o',
 24: 'p',
 25: 'q',
 26: 'r',
 27: 's',
 28: 't',
 29: 'u',
 30: 'v',
 31: 'w',
 32: 'y',
 33: 'z'}

In [7]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    # Loop untill we recieve a stop sign
    while not stop_condition:
        # Get output and internal states of the decoder 
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Get the predicted token (the token with the highest score)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Get the character belonging to the token
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Append char to output
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [8]:
def lettoword(let):
    my_dict = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9','4':'a', 'E': 'b', 'I': 'c', '(+)': 'd', '(-)': 'e', "Y'": 'f', 'abs': 'g', 'acos': 'h', 'acosh': 'i', '+': 'k', 'asin': 'l', 'asinh': 'm', 'atan': 'n', 'atanh': 'o', 'cos': 'p', 'cosh': 'q', '/': 'r', 'exp': 's', 'ln': 't', '*': 'u', 'pi': 'v', 'pow': 'w', 'sign': 'x', 'sin': 'y', 'sinh': 'z', 'sqrt': 'A', '-': 'B', 'tan': 'C', 'tanh': 'D', 'x': 'E'}
    rev = {v:k for k, v in my_dict.items()}
    let = list(let)
    ret = []
    for key in let:
        ret.append(rev[key])
    return " ".join(ret)
        
def wordtolet(word):
    my_dict = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '4':'a', 'E': 'b', 'I': 'c', '(+)': 'd', '(-)': 'e', "Y'": 'f', 'abs': 'g', 'acos': 'h', 'acosh': 'i', '+': 'k', 'asin': 'l', 'asinh': 'm', 'atan': 'n', 'atanh': 'o', 'cos': 'p', 'cosh': 'q', '/': 'r', 'exp': 's', 'ln': 't', '*': 'u', 'pi': 'v', 'pow': 'w', 'sign': 'x', 'sin': 'y', 'sinh': 'z', 'sqrt': 'A', '-': 'B', 'tan': 'C', 'tanh': 'D', 'x': 'E'}
    word = word.split()
    ret = []
    for key in word:
        ret.append(my_dict[key])
    return "".join(ret)

In [9]:
def solve(text):
    my_text = wordtolet(text)
    placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))
    for i, char in enumerate(my_text):
        #print(i,char, input_token_index[char])
        placeholder[0,i,input_token_index[char]] = 1
    sol = decode_sequence(placeholder)[:-1]
    sol = lettoword(sol)
    return sol

In [16]:
def print_examples():
    train_lines = open('minialpha.train').read().split('\n')
    for train_line in train_lines[:1]:
        train_input_eq, train_target_eq = train_line.split('\t')
        train_input_eq = lettoword(train_input_eq)
        train_target_eq = lettoword(train_target_eq)
        sol = solve(train_input_eq)
        #if(solve(train_input_eq) == train_target_eq):
        print("\nInput: " + train_input_eq + "\nPrediction: " + lettoword(wordtolet(sol)[:min(len(wordtolet(sol)), len(wordtolet(train_target_eq)))]) + "\nTarget: " + train_target_eq)
        

In [11]:
from keras.models import load_model
import numpy as np

In [13]:
encoder_model = load_model('int_enc.tf')

In [14]:
decoder_model = load_model('int_dec.tf')

In [20]:
print_examples()


Input: - Y' + x pow x (+) 2
Prediction: + * / (+) 1 (+) 2 pow x (+) 2 + * / (+) 1 (+) 4 pow x (+)
Target: + * / (+) 1 (+) 2 pow x (+) 2 * / (+) 1 (+) 3 pow x (+) 3
